# Kaggle Home Credit Default Risk - Bureau Exploration

In [1]:
import os
import sys
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis, iqr

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [45]:
app_train = pd.read_csv('../data/application_train.csv')
print('application_train data shape: ', app_train.shape)
app_test = pd.read_csv('../data/application_test.csv')
print('Testing data shape: ', app_test.shape)

application_train data shape:  (307511, 122)
Testing data shape:  (48744, 121)


In [2]:
df_burb = pd.read_csv('../data/bureau_balance.csv')
print('bureau_balance data shape: ', df_burb.shape)
df_bur = pd.read_csv('../data/bureau.csv')
print('bureau data shape: ', df_bur.shape)

bureau_balance data shape:  (27299925, 3)
bureau data shape:  (1716428, 17)


###### first step

In [3]:
%%time

df_burb_d = pd.get_dummies(df_burb)

for col in df_burb_d.columns:
    if 'STATUS' in col:
        df_burb_d[col] = df_burb_d[col] * df_burb_d['MONTHS_BALANCE']

# Create Step1 Aggregation Strategy
# Functions tested : [np.min, np.max, np.mean, np.sum, 'median', 'count', 'var', np.std, skew, kurtosis, iqr]
aggregation_strategy_step1 = {}
for col in bureau_balance.columns:
    if 'SK_ID' not in col:
        if 'MONTHS' in col:
            aggregation_strategy_step1[col] = [np.min, np.max, np.mean, np.sum, 'median', 'count', 'var', np.std]
        elif 'STATUS' in col:
            aggregation_strategy_step1[col] = [np.min, np.max, np.mean, np.sum, 'median', 'var', np.std]
        else:
            aggregation_strategy_step1[col] = [np.min, np.max, np.mean, np.sum, 'median', 'var']

            
df_burb_d_agg = df_burb_d.groupby(['SK_ID_BUREAU'], as_index=False).agg(aggregation_strategy_step1)

df_burb_d_agg.columns = [col[0] + "_" + col[1] for col in df_burb_d_agg.columns]
df_burb_d_agg.rename(columns={'SK_ID_BUREAU_':'SK_ID_BUREAU'}, inplace=True)

print(df_bur.shape)
print(df_burb_d_agg.shape)
df_bur_conso = df_bur.merge(df_burb_d_agg, on='SK_ID_BUREAU', how='left')
print(df_bur_conso.shape)

df_bur_conso.head()

(1716428, 17)
(817395, 56)
(1716428, 72)
Wall time: 1min 20s


In [23]:
print(df_bur_conso.shape)
df_bur_conso_d = pd.get_dummies(df_bur_conso)
print(df_bur_conso_d.shape)

curr_list = []
for col in df_bur_conso_d.columns:
    if 'CREDIT_CURRENCY' in col:
        df_bur_conso_d[col] = df_bur_conso_d[col] * df_bur_conso_d['AMT_CREDIT_SUM']
        curr_list.append(col)
    if 'CREDIT_ACTIVE' in col:
        df_bur_conso_d[col] = df_bur_conso_d[col] * df_bur_conso_d['AMT_CREDIT_SUM']
        curr_list.append(col)
        
df_bur_conso_d[curr_list].head()

(1716428, 72)
(1716428, 92)


,CREDIT_ACTIVE_Active,CREDIT_ACTIVE_Bad debt,CREDIT_ACTIVE_Closed,CREDIT_ACTIVE_Sold,CREDIT_CURRENCY_currency 1,CREDIT_CURRENCY_currency 2,CREDIT_CURRENCY_currency 3,CREDIT_CURRENCY_currency 4
0,0.0,0.0,91323.0,0.0,91323.0,0.0,0.0,0.0
1,225000.0,0.0,0.0,0.0,225000.0,0.0,0.0,0.0
2,464323.5,0.0,0.0,0.0,464323.5,0.0,0.0,0.0
3,90000.0,0.0,0.0,0.0,90000.0,0.0,0.0,0.0
4,2700000.0,0.0,0.0,0.0,2700000.0,0.0,0.0,0.0


In [25]:
aggregation_strategy_dev = {}

for col in df_bur_conso_d.columns:
    if 'SK_ID' not in col:
        if 'MONTHS' in col:
            aggregation_strategy_dev[col] = [np.min, np.max, np.mean, np.sum, 'median', 'var']
        elif 'STATUS' in col:
            aggregation_strategy_dev[col] = [np.min, np.max, np.mean, np.sum, 'median', 'var']
        elif 'CREDIT' in col:
            aggregation_strategy_dev[col] = [np.min, np.max, np.mean, np.sum, 'median', 'var']
        else:
            aggregation_strategy_dev[col] = [np.min, np.max, np.mean, np.sum, 'median', 'var']


In [26]:
%%time

df_bur_conso_d_agg = df_bur_conso_d.groupby(['SK_ID_CURR'], as_index=False).agg(aggregation_strategy_dev)

df_bur_conso_d_agg.columns = [col[0] + "_" + col[1] for col in df_bur_conso_d_agg.columns]
df_bur_conso_d_agg.rename(columns={'SK_ID_BUREAU_':'SK_ID_BUREAU', 'SK_ID_CURR_':'SK_ID_CURR'}, inplace=True)

Wall time: 1min 22s


In [31]:
df_bur_conso_d_agg.shape

(305811, 541)

In [48]:
df_bur_conso_d_agg[df_bur_conso_d_agg['SK_ID_CURR'].duplicated()]['SK_ID_CURR']

Series([], Name: SK_ID_CURR, dtype: int64)

In [ ]:
print(app_train.shape)
app_train_bur = app_train.merge(df_bur_conso_d_agg, on='SK_ID_CURR', how='left')
print(app_train_bur.shape)

In [30]:
coltmp = ['SK_ID_CURR']
for col in df_bur_conso_d_agg.columns:
    if 'MONTHS_BALANCE' in col:
        coltmp.append(col)
    
df_bur_conso_d_agg[coltmp]

,SK_ID_CURR,MONTHS_BALANCE_amin_amin,MONTHS_BALANCE_amin_amax,MONTHS_BALANCE_amin_mean,MONTHS_BALANCE_amin_sum,MONTHS_BALANCE_amin_median,MONTHS_BALANCE_amin_var,MONTHS_BALANCE_amax_amin,MONTHS_BALANCE_amax_amax,MONTHS_BALANCE_amax_mean,MONTHS_BALANCE_amax_sum,MONTHS_BALANCE_amax_median,MONTHS_BALANCE_amax_var,MONTHS_BALANCE_mean_amin,MONTHS_BALANCE_mean_amax,MONTHS_BALANCE_mean_mean,MONTHS_BALANCE_mean_sum,MONTHS_BALANCE_mean_median,MONTHS_BALANCE_mean_var,MONTHS_BALANCE_sum_amin,MONTHS_BALANCE_sum_amax,MONTHS_BALANCE_sum_mean,MONTHS_BALANCE_sum_sum,MONTHS_BALANCE_sum_median,MONTHS_BALANCE_sum_var,MONTHS_BALANCE_median_amin,MONTHS_BALANCE_median_amax,MONTHS_BALANCE_median_mean,MONTHS_BALANCE_median_sum,MONTHS_BALANCE_median_median,MONTHS_BALANCE_median_var,MONTHS_BALANCE_count_amin,MONTHS_BALANCE_count_amax,MONTHS_BALANCE_count_mean,MONTHS_BALANCE_count_sum,MONTHS_BALANCE_count_median,MONTHS_BALANCE_count_var,MONTHS_BALANCE_var_amin,MONTHS_BALANCE_var_amax,MONTHS_BALANCE_var_mean,MONTHS_BALANCE_var_sum,MONTHS_BALANCE_var_median,MONTHS_BALANCE_var_var
0,100001,-51.0,-1.0,-23.571429,-165.0,-28.0,257.619048,0.0,0.0,0.000000,0.0,0.0,0.000000,-25.5,-0.5,-11.785714,-82.5,-14.00,64.404762,-1326.0,-1.0,-400.000000,-2800.0,-406.0,1.982400e+05,-25.5,-0.5,-11.785714,-82.5,-14.00,64.404762,2.0,52.0,24.571429,172.0,29.0,257.619048,0.500000,229.666667,70.761905,495.333333,72.500000,5889.804233
1,100002,-47.0,-3.0,-28.250000,-226.0,-34.0,200.500000,-32.0,0.0,-15.500000,-124.0,-18.5,116.285714,-39.5,-1.5,-21.875000,-175.0,-26.00,148.267857,-632.0,-6.0,-337.625000,-2701.0,-424.0,5.574684e+04,-39.5,-1.5,-21.875000,-175.0,-26.00,148.267857,4.0,22.0,13.750000,110.0,16.0,40.500000,1.666667,42.166667,19.854167,158.833333,22.666667,171.281250
2,100003,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
3,100004,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
4,100005,-12.0,-2.0,-6.000000,-18.0,-4.0,28.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,-6.0,-1.0,-3.000000,-9.0,-2.00,7.000000,-78.0,-3.0,-30.333333,-91.0,-10.0,1.716333e+03,-6.0,-1.0,-3.000000,-9.0,-2.00,7.000000,3.0,13.0,7.000000,21.0,5.0,28.000000,1.000000,15.166667,6.222222,18.666667,2.500000,60.564815
5,100007,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
6,100008,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
7,100009,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
8,100010,-90.0,-37.0,-63.500000,-127.0,-63.5,1404.500000,-55.0,-2.0,-28.500000,-57.0,-28.5,1404.500000,-72.5,-19.5,-46.000000,-92.0,-46.00,1404.500000,-2610.0,-702.0,-1656.000000,-3312.0,-1656.0,1.820232e+06,-72.5,-19.5,-46.000000,-92.0,-46.00,1404.500000,36.0,36.0,36.000000,72.0,36.0,0.000000,111.000000,111.000000,111.000000,222.000000,111.000000,0.000000
9,100011,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN


###### Next step

In [6]:
print(df_bur.shape)
df_bur_d = pd.get_dummies(df_bur)
print(df_bur_d.shape)

(1716428, 17)
(1716428, 37)


In [7]:
curr_list = []
for col in df_bur_d.columns:
    if 'CREDIT_CURRENCY' in col:
        df_bur_d[col] = df_bur_d[col] * df_bur_d['AMT_CREDIT_SUM']
        curr_list.append(col)
        
df_bur_d[curr_list].head()

,CREDIT_CURRENCY_currency 1,CREDIT_CURRENCY_currency 2,CREDIT_CURRENCY_currency 3,CREDIT_CURRENCY_currency 4
0,91323.0,0.0,0.0,0.0
1,225000.0,0.0,0.0,0.0
2,464323.5,0.0,0.0,0.0
3,90000.0,0.0,0.0,0.0
4,2700000.0,0.0,0.0,0.0


In [9]:
%%time

aggregation_strategy = {
    'DAYS_CREDIT': [np.min, np.max, np.mean, np.sum, 'median', 'count', 'var', np.std, skew, kurtosis, iqr], 
    'CREDIT_DAY_OVERDUE': [np.min, np.max, np.mean, np.sum, 'median', 'count', 'var', np.std, skew, kurtosis, iqr],
    'DAYS_CREDIT_ENDDATE': [np.min, np.max, np.mean, np.sum, 'median', 'count', 'var', np.std, skew, kurtosis, iqr],
    'DAYS_ENDDATE_FACT': [np.min, np.max, np.mean, np.sum, 'median', 'count', 'var', np.std, skew, kurtosis, iqr],
    'AMT_CREDIT_MAX_OVERDUE': [np.min, np.max, np.mean, np.sum, 'median', 'count', 'var', np.std, skew, kurtosis, iqr],
    'CNT_CREDIT_PROLONG': [np.min, np.max, np.mean, np.sum, 'median', 'count', 'var', np.std, skew, kurtosis, iqr],
    'AMT_CREDIT_SUM': [np.min, np.max, np.mean, np.sum, 'median', 'count', 'var', np.std, skew, kurtosis, iqr],
    'AMT_CREDIT_SUM_DEBT': [np.min, np.max, np.mean, np.sum, 'median', 'count', 'var', np.std, skew, kurtosis, iqr],
    'AMT_CREDIT_SUM_LIMIT': [np.min, np.max, np.mean, np.sum, 'median', 'count', 'var', np.std, skew, kurtosis, iqr],
    'AMT_CREDIT_SUM_OVERDUE': [np.min, np.max, np.mean, np.sum, 'median', 'count', 'var', np.std, skew, kurtosis, iqr],
    'DAYS_CREDIT_UPDATE': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'AMT_ANNUITY': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'CREDIT_ACTIVE_Active': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'CREDIT_ACTIVE_Bad debt': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'CREDIT_ACTIVE_Closed': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'CREDIT_ACTIVE_Sold': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'CREDIT_CURRENCY_currency 1': [np.min, np.max, np.mean, np.sum, 'median', 'count', 'var', np.std, skew, kurtosis, iqr],
    'CREDIT_CURRENCY_currency 2': [np.min, np.max, np.mean, np.sum, 'median', 'count', 'var', np.std, skew, kurtosis, iqr],
    'CREDIT_CURRENCY_currency 3': [np.min, np.max, np.mean, np.sum, 'median', 'count', 'var', np.std, skew, kurtosis, iqr],
    'CREDIT_CURRENCY_currency 4': [np.min, np.max, np.mean, np.sum, 'median', 'count', 'var', np.std, skew, kurtosis, iqr],
    'CREDIT_TYPE_Another type of loan': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'CREDIT_TYPE_Car loan': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'CREDIT_TYPE_Cash loan (non-earmarked)': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'CREDIT_TYPE_Consumer credit': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'CREDIT_TYPE_Credit card': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'CREDIT_TYPE_Interbank credit': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'CREDIT_TYPE_Loan for business development': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'CREDIT_TYPE_Loan for purchase of shares (margin lending)': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'CREDIT_TYPE_Loan for the purchase of equipment': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'CREDIT_TYPE_Loan for working capital replenishment': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'CREDIT_TYPE_Microloan': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'CREDIT_TYPE_Mobile operator loan': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'CREDIT_TYPE_Mortgage': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'CREDIT_TYPE_Real estate loan': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'CREDIT_TYPE_Unknown type of loan': [np.min, np.max, np.mean, np.sum, 'median', 'var']
    }


aggregation_strategy_dev = {
    'DAYS_CREDIT': [np.min, np.max, np.mean, np.sum, 'median', 'count', 'var'], 
    'CREDIT_DAY_OVERDUE': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'DAYS_CREDIT_ENDDATE': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'DAYS_ENDDATE_FACT': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'AMT_CREDIT_MAX_OVERDUE': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'CNT_CREDIT_PROLONG': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'AMT_CREDIT_SUM': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'AMT_CREDIT_SUM_DEBT': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'AMT_CREDIT_SUM_LIMIT': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'AMT_CREDIT_SUM_OVERDUE': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'DAYS_CREDIT_UPDATE': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'AMT_ANNUITY': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'CREDIT_ACTIVE_Active': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'CREDIT_ACTIVE_Bad debt': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'CREDIT_ACTIVE_Closed': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'CREDIT_ACTIVE_Sold': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'CREDIT_CURRENCY_currency 1': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'CREDIT_CURRENCY_currency 2': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'CREDIT_CURRENCY_currency 3': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'CREDIT_CURRENCY_currency 4': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'CREDIT_TYPE_Another type of loan': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'CREDIT_TYPE_Car loan': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'CREDIT_TYPE_Cash loan (non-earmarked)': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'CREDIT_TYPE_Consumer credit': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'CREDIT_TYPE_Credit card': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'CREDIT_TYPE_Interbank credit': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'CREDIT_TYPE_Loan for business development': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'CREDIT_TYPE_Loan for purchase of shares (margin lending)': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'CREDIT_TYPE_Loan for the purchase of equipment': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'CREDIT_TYPE_Loan for working capital replenishment': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'CREDIT_TYPE_Microloan': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'CREDIT_TYPE_Mobile operator loan': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'CREDIT_TYPE_Mortgage': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'CREDIT_TYPE_Real estate loan': [np.min, np.max, np.mean, np.sum, 'median', 'var'],
    'CREDIT_TYPE_Unknown type of loan': [np.min, np.max, np.mean, np.sum, 'median', 'var']
    }

df_bur_d_agg = df_bur_d.groupby(['SK_ID_CURR', 'SK_ID_BUREAU'], as_index=False).agg(aggregation_strategy_dev)

df_bur_d_agg.columns = [col[0] + "_" + col[1] for col in df_bur_d_agg.columns]
df_bur_d_agg.rename(columns={'SK_ID_BUREAU_':'SK_ID_BUREAU', 'SK_ID_CURR_':'SK_ID_CURR'}, inplace=True)

Wall time: 1min 27s


In [13]:
df_bur_d_agg.head()

,SK_ID_CURR,SK_ID_BUREAU,DAYS_CREDIT_amin,DAYS_CREDIT_amax,DAYS_CREDIT_mean,DAYS_CREDIT_sum,DAYS_CREDIT_median,DAYS_CREDIT_count,DAYS_CREDIT_var,CREDIT_DAY_OVERDUE_amin,CREDIT_DAY_OVERDUE_amax,CREDIT_DAY_OVERDUE_mean,CREDIT_DAY_OVERDUE_sum,CREDIT_DAY_OVERDUE_median,CREDIT_DAY_OVERDUE_var,DAYS_CREDIT_ENDDATE_amin,DAYS_CREDIT_ENDDATE_amax,DAYS_CREDIT_ENDDATE_mean,DAYS_CREDIT_ENDDATE_sum,DAYS_CREDIT_ENDDATE_median,DAYS_CREDIT_ENDDATE_var,DAYS_ENDDATE_FACT_amin,DAYS_ENDDATE_FACT_amax,DAYS_ENDDATE_FACT_mean,DAYS_ENDDATE_FACT_sum,DAYS_ENDDATE_FACT_median,DAYS_ENDDATE_FACT_var,AMT_CREDIT_MAX_OVERDUE_amin,AMT_CREDIT_MAX_OVERDUE_amax,AMT_CREDIT_MAX_OVERDUE_mean,AMT_CREDIT_MAX_OVERDUE_sum,AMT_CREDIT_MAX_OVERDUE_median,AMT_CREDIT_MAX_OVERDUE_var,CNT_CREDIT_PROLONG_amin,CNT_CREDIT_PROLONG_amax,CNT_CREDIT_PROLONG_mean,CNT_CREDIT_PROLONG_sum,CNT_CREDIT_PROLONG_median,CNT_CREDIT_PROLONG_var,AMT_CREDIT_SUM_amin,AMT_CREDIT_SUM_amax,AMT_CREDIT_SUM_mean,AMT_CREDIT_SUM_sum,AMT_CREDIT_SUM_median,AMT_CREDIT_SUM_var,AMT_CREDIT_SUM_DEBT_amin,AMT_CREDIT_SUM_DEBT_amax,AMT_CREDIT_SUM_DEBT_mean,AMT_CREDIT_SUM_DEBT_sum,AMT_CREDIT_SUM_DEBT_median,AMT_CREDIT_SUM_DEBT_var,AMT_CREDIT_SUM_LIMIT_amin,AMT_CREDIT_SUM_LIMIT_amax,AMT_CREDIT_SUM_LIMIT_mean,AMT_CREDIT_SUM_LIMIT_sum,AMT_CREDIT_SUM_LIMIT_median,AMT_CREDIT_SUM_LIMIT_var,AMT_CREDIT_SUM_OVERDUE_amin,AMT_CREDIT_SUM_OVERDUE_amax,AMT_CREDIT_SUM_OVERDUE_mean,AMT_CREDIT_SUM_OVERDUE_sum,AMT_CREDIT_SUM_OVERDUE_median,AMT_CREDIT_SUM_OVERDUE_var,DAYS_CREDIT_UPDATE_amin,DAYS_CREDIT_UPDATE_amax,DAYS_CREDIT_UPDATE_mean,DAYS_CREDIT_UPDATE_sum,DAYS_CREDIT_UPDATE_median,DAYS_CREDIT_UPDATE_var,AMT_ANNUITY_amin,AMT_ANNUITY_amax,AMT_ANNUITY_mean,AMT_ANNUITY_sum,AMT_ANNUITY_median,AMT_ANNUITY_var,CREDIT_ACTIVE_Active_amin,CREDIT_ACTIVE_Active_amax,CREDIT_ACTIVE_Active_mean,CREDIT_ACTIVE_Active_sum,CREDIT_ACTIVE_Active_median,CREDIT_ACTIVE_Active_var,CREDIT_ACTIVE_Bad debt_amin,CREDIT_ACTIVE_Bad debt_amax,CREDIT_ACTIVE_Bad debt_mean,CREDIT_ACTIVE_Bad debt_sum,CREDIT_ACTIVE_Bad debt_median,CREDIT_ACTIVE_Bad debt_var,CREDIT_ACTIVE_Closed_amin,CREDIT_ACTIVE_Closed_amax,CREDIT_ACTIVE_Closed_mean,CREDIT_ACTIVE_Closed_sum,CREDIT_ACTIVE_Closed_median,CREDIT_ACTIVE_Closed_var,CREDIT_ACTIVE_Sold_amin,CREDIT_ACTIVE_Sold_amax,CREDIT_ACTIVE_Sold_mean,CREDIT_ACTIVE_Sold_sum,CREDIT_ACTIVE_Sold_median,CREDIT_ACTIVE_Sold_var,CREDIT_CURRENCY_currency 1_amin,CREDIT_CURRENCY_currency 1_amax,CREDIT_CURRENCY_currency 1_mean,CREDIT_CURRENCY_currency 1_sum,CREDIT_CURRENCY_currency 1_median,CREDIT_CURRENCY_currency 1_var,CREDIT_CURRENCY_currency 2_amin,CREDIT_CURRENCY_currency 2_amax,CREDIT_CURRENCY_currency 2_mean,CREDIT_CURRENCY_currency 2_sum,CREDIT_CURRENCY_currency 2_median,CREDIT_CURRENCY_currency 2_var,CREDIT_CURRENCY_currency 3_amin,CREDIT_CURRENCY_currency 3_amax,CREDIT_CURRENCY_currency 3_mean,CREDIT_CURRENCY_currency 3_sum,CREDIT_CURRENCY_currency 3_median,CREDIT_CURRENCY_currency 3_var,CREDIT_CURRENCY_currency 4_amin,CREDIT_CURRENCY_currency 4_amax,CREDIT_CURRENCY_currency 4_mean,CREDIT_CURRENCY_currency 4_sum,CREDIT_CURRENCY_currency 4_median,CREDIT_CURRENCY_currency 4_var,CREDIT_TYPE_Another type of loan_amin,CREDIT_TYPE_Another type of loan_amax,CREDIT_TYPE_Another type of loan_mean,CREDIT_TYPE_Another type of loan_sum,CREDIT_TYPE_Another type of loan_median,CREDIT_TYPE_Another type of loan_var,CREDIT_TYPE_Car loan_amin,CREDIT_TYPE_Car loan_amax,CREDIT_TYPE_Car loan_mean,CREDIT_TYPE_Car loan_sum,CREDIT_TYPE_Car loan_median,CREDIT_TYPE_Car loan_var,CREDIT_TYPE_Cash loan (non-earmarked)_amin,CREDIT_TYPE_Cash loan (non-earmarked)_amax,CREDIT_TYPE_Cash loan (non-earmarked)_mean,CREDIT_TYPE_Cash loan (non-earmarked)_sum,CREDIT_TYPE_Cash loan (non-earmarked)_median,CREDIT_TYPE_Cash loan (non-earmarked)_var,CREDIT_TYPE_Consumer credit_amin,CREDIT_TYPE_Consumer credit_amax,CREDIT_TYPE_Consumer credit_mean,CREDIT_TYPE_Consumer credit_sum,CREDIT_TYPE_Consumer credit_median,CREDIT_TYPE_Consumer credit_var,CREDIT_TYPE_Credit card_amin,CREDIT_TYP

In [67]:
df_bur_d[df_bur_d['SK_ID_CURR'] == 215354]

,SK_ID_CURR,SK_ID_BUREAU,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,CREDIT_ACTIVE_Active,CREDIT_ACTIVE_Bad debt,CREDIT_ACTIVE_Closed,CREDIT_ACTIVE_Sold,CREDIT_CURRENCY_currency 1,CREDIT_CURRENCY_currency 2,CREDIT_CURRENCY_currency 3,CREDIT_CURRENCY_currency 4,CREDIT_TYPE_Another type of loan,CREDIT_TYPE_Car loan,CREDIT_TYPE_Cash loan (non-earmarked),CREDIT_TYPE_Consumer credit,CREDIT_TYPE_Credit card,CREDIT_TYPE_Interbank credit,CREDIT_TYPE_Loan for business development,CREDIT_TYPE_Loan for purchase of shares (margin lending),CREDIT_TYPE_Loan for the purchase of equipment,CREDIT_TYPE_Loan for working capital replenishment,CREDIT_TYPE_Microloan,CREDIT_TYPE_Mobile operator loan,CREDIT_TYPE_Mortgage,CREDIT_TYPE_Real estate loan,CREDIT_TYPE_Unknown type of loan
0,215354,5714462,-497,0,-153.0,-153.0,NaN,0,91323.0,0.00,NaN,0.0,-131,NaN,0,0,1,0,91323.0,0.0,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,215354,5714463,-208,0,1075.0,NaN,NaN,0,225000.0,171342.00,NaN,0.0,-20,NaN,1,0,0,0,225000.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,215354,5714464,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,-16,NaN,1,0,0,0,464323.5,0.0,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,215354,5714465,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,-16,NaN,1,0,0,0,90000.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,215354,5714466,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,-21,NaN,1,0,0,0,2700000.0,0.0,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
5,215354,5714467,-273,0,27460.0,NaN,0.0,0,180000.0,71017.38,108982.62,0.0,-31,NaN,1,0,0,0,180000.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
6,215354,5714468,-43,0,79.0,NaN,0.0,0,42103.8,42103.80,0.00,0.0,-22,NaN,1,0,0,0,42103.8,0.0,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
225157,215354,5714458,-1872,0,-776.0,-1203.0,NaN,0,494550.0,0.00,0.00,0.0,-696,NaN,0,0,1,0,494550.0,0.0,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
225158,215354,5714459,-1734,0,-638.0,-1336.0,NaN,0,1084032.0,0.00,NaN,0.0,-1336,NaN,0,0,1,0,1084032.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
225159,215354,5714460,-1333,0,-1089.0,-987.0,NaN,0,102150.0,NaN,NaN,0.0,-984,NaN,0,0,1,0,102150.0,0.0,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [45]:
df_bur[df_bur['SK_ID_CURR'] == 215354]

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.00,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.00,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN
5,215354,5714467,Active,currency 1,-273,0,27460.0,NaN,0.0,0,180000.0,71017.38,108982.62,0.0,Credit card,-31,NaN
6,215354,5714468,Active,currency 1,-43,0,79.0,NaN,0.0,0,42103.8,42103.80,0.00,0.0,Consumer credit,-22,NaN
225157,215354,5714458,Closed,currency 1,-1872,0,-776.0,-1203.0,NaN,0,494550.0,0.00,0.00,0.0,Consumer credit,-696,NaN
225158,215354,5714459,Closed,currency 1,-1734,0,-638.0,-1336.0,NaN,0,1084032.0,0.00,NaN,0.0,Car loan,-1336,NaN
225159,215354,5714460,Closed,currency 1,-1333,0,-1089.0,-987.0,NaN,0,102150.0,NaN,NaN,0.0,Consumer credit,-984,NaN


In [41]:
app_train[app_train['SK_ID_CURR'] == 215354].count()

SK_ID_CURR                    1
TARGET                        1
NAME_CONTRACT_TYPE            1
CODE_GENDER                   1
FLAG_OWN_CAR                  1
FLAG_OWN_REALTY               1
CNT_CHILDREN                  1
AMT_INCOME_TOTAL              1
AMT_CREDIT                    1
AMT_ANNUITY                   1
AMT_GOODS_PRICE               1
NAME_TYPE_SUITE               1
NAME_INCOME_TYPE              1
NAME_EDUCATION_TYPE           1
NAME_FAMILY_STATUS            1
NAME_HOUSING_TYPE             1
REGION_POPULATION_RELATIVE    1
DAYS_BIRTH                    1
DAYS_EMPLOYED                 1
DAYS_REGISTRATION             1
DAYS_ID_PUBLISH               1
OWN_CAR_AGE                   1
FLAG_MOBIL                    1
FLAG_EMP_PHONE                1
FLAG_WORK_PHONE               1
FLAG_CONT_MOBILE              1
FLAG_PHONE                    1
FLAG_EMAIL                    1
OCCUPATION_TYPE               1
CNT_FAM_MEMBERS               1
                             ..
DEF_30_C

In [44]:
days_col = []
for col in app_train.columns:
    if 'DAY' in col:
        days_col.append(col)
        
app_train[app_train['SK_ID_CURR'] == 215354][days_col]

,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,WEEKDAY_APPR_PROCESS_START,DAYS_LAST_PHONE_CHANGE,AMT_REQ_CREDIT_BUREAU_DAY
99355,-17520,-939,-1203.0,-1064,MONDAY,-1553.0,0.0


In [46]:
_days_col = []
for col in df_bur.columns:
    if 'DAY' in col:
        _days_col.append(col)
        
df_bur[df_bur['SK_ID_CURR'] == 215354][_days_col]

,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,DAYS_CREDIT_UPDATE
0,-497,0,-153.0,-153.0,-131
1,-208,0,1075.0,NaN,-20
2,-203,0,528.0,NaN,-16
3,-203,0,NaN,NaN,-16
4,-629,0,1197.0,NaN,-21
5,-273,0,27460.0,NaN,-31
6,-43,0,79.0,NaN,-22
225157,-1872,0,-776.0,-1203.0,-696
225158,-1734,0,-638.0,-1336.0,-1336
225159,-1333,0,-1089.0,-987.0,-984
